## <font color=blue> Lec3 TCP --- Client</font>   
-----------------------------------------------------------

#### TCP Socket: Client

In [2]:
import socket
import datetime

In [2]:
host = '127.0.0.1'
port = 1060

In [30]:
host = '192.168.163.131'
port = 5151

In [3]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

In [47]:
sock.connect((host, port))

In [48]:
print('Client has been assigned socket name', sock.getsockname())

Client has been assigned socket name ('127.0.0.1', 25163)


In [52]:
sock.send(b'hello')

5

In [13]:
sock.send(b'world')

5

In [34]:
sock.close()

#### sock tools

In [3]:
#####################################################
# 常用的socket 读写函数
def sock_readline(sock):
    line = b''
    data = b''
    while (data != b'\n'):
        data = sock.recv(1)
        if (data == b''):
            break
        line += data
    return line

def sock_readn(sock, length):
    data = b''
    while len(data) < length:
        more = sock.recv(length - len(data))
        if not more:
            return data
        data += more
    return data

def sock_writeline(sock, data):
    sock.send(data)
    sock.send(b'\n')

def sock_writen(sock, data, length):
    if (len(data) > length):
        sock.send(data[0:length])
    else:
        sock.send(data)

def sock_write(sock, data):
    sock.send(data)

############################################################
# 提供带buffer缓存的sock读写函数，减少socket函数调用次数，提高效率

def sock_buf_readline(sock, data_buf=b''):
    data_list = []
    index = -1
    data = b''
    if (data_buf != b''):
        data = data_buf
        index = data.find(b'\n')
    
    while (index == -1):
        if (data != b''):
            data_list.append(data)
        
        data = sock.recv(4096)
        if (data == b''):
            break
        
        index = data.find(b'\n')
    
    if (data == b''):
        return (b''.join(data_list), b'')
    
    data_list.append(data[0:index+1])
    return (b''.join(data_list), data[index+1:])

def sock_buf_readn(sock, length, data_buf=b''):
    data_list = []
    
    data = data_buf
    index = len(data_buf)
    while (index < length):
        if (data != b''):
            data_list.append(data)
        
        data = sock.recv(4096)
        if (data == b''):
            break
        
        index += len(data)
    
    if (data == b''):
        return (b''.join(data_list), b'')
    
    index = len(data) - (length - index)
    data_list.append(data[0:index])
    return (b''.join(data_list), data[index:])

def sock_buf_read(sock, data_buf=b''):
    if (len(data_buf) > 0):
        return data_buf
       
    return sock.recv(4096)

#### 定长消息

In [9]:
PACKET_SIZE = 250

In [7]:
def sendPacket(sock, data):
    sock.sendall(data)
    sock.sendall(b' ' * (PACKET_SIZE - len(data)))

In [8]:
def recvPacket(sock):
    data = sock_readn(sock, PACKET_SIZE)
    if (len(data) != PACKET_SIZE):
        return b''
    data = data.rstrip()
    return data

In [4]:
import socket
import datetime

In [5]:
host = '127.0.0.1'
port = 5151

In [6]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((host, port))
print('Client has been assigned socket name', sock.getsockname())

Client has been assigned socket name ('127.0.0.1', 54575)


In [10]:
sendPacket(sock, b"hello world")

In [12]:
recvPacket(sock)

b'farewell!'

In [13]:
sock.close()

#### 长度前缀--文本长度

In [14]:
def sendPacket(sock, data):
    len_str = "{:05d}".format(len(data))
    sock.sendall(len_str.encode("utf8") + data)

In [19]:
def recvPacket(sock):
    data = b''
    len_data = sock_readn(sock, 5)
    if (len(len_data) < 5):
        return data
    length = int(len_data.decode("utf8"))
    data = sock_readn(sock, length)
    return data

In [4]:
import socket
import datetime

In [23]:
host = '127.0.0.1'
port = 5151
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((host, port))
print('Client has been assigned socket name', sock.getsockname())

Client has been assigned socket name ('127.0.0.1', 57794)


In [24]:
sendPacket(sock, b"hello world")

In [25]:
recvPacket(sock)

b'farewell!'

In [26]:
sock.close()

#### 长度前缀--二进制长度

In [28]:
hex(4253)

'0x109d'

In [27]:
import struct

In [33]:
#little endian
struct.pack('<i', 4253)

b'\x9d\x10\x00\x00'

In [31]:
#big endian
struct.pack('>i', 4253)

b'\x00\x00\x10\x9d'

In [35]:
struct.unpack('<i', b'\x9d\x10\x00\x00')

(4253,)

In [36]:
struct.unpack('>i', b'\x9d\x10\x00\x00')

(-1659895808,)

#### socket库提供了字节序转换函数 ntohl, ntohs, htonl, htons

In [38]:
#network (= big-endian)
struct.pack('!i', 4253)

b'\x00\x00\x10\x9d'

In [4]:
def sendPacket(sock, data):
    sock.sendall(struct.pack('!h', len(data)) + data)

In [11]:
def recvPacket(sock):
    data = b''
    len_data = sock_readn(sock, 2)
    if (len(len_data) < 2):
        return data
    length, = struct.unpack('!h', len_data)
    data = sock_readn(sock, length)
    return data

In [9]:
import socket
import datetime
import struct

In [7]:
host = '127.0.0.1'
port = 5151
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((host, port))
print('Client has been assigned socket name', sock.getsockname())

Client has been assigned socket name ('127.0.0.1', 63814)


In [10]:
sendPacket(sock, b"hello world")

In [12]:
recvPacket(sock)

b'farewell!'

In [13]:
sock.close()

模式一

In [15]:
address = ("127.0.0.1", 5150)

In [19]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(address)

In [ ]:
sock.shutdown(socket.SHUT_RD)

In [20]:
sock.sendall(b'Beautiful is better than ugly.\n')
sock.sendall(b'Explicit is better than implicit.\n')
sock.sendall(b'Simple is better than complex.\n')

In [21]:
sock.shutdown(socket.SHUT_WR)

In [18]:
sock.close()

#### pickle格式

In [22]:
import pickle

In [26]:
address = ("127.0.0.1", 5150)

In [27]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(address)

In [30]:
sock.sendall(pickle.dumps([5, 6, 7]))

In [31]:
info = {"username":"zhangsan", "age":"18", "score":[80, 98, 68]}
sock.sendall(pickle.dumps(info))

In [32]:
sock.close()